In [1]:
# This notebook contains a naive mlp for garbage classification
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import utils
from vocab import Vocabulary
import pandas as pd
import numpy as np
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader

/home/bowen/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/bowen/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
import os
EMB_DIM = 100 # embedding dimension to be used, choose from [50, 100, 200, 300]
glove_path = os.path.expanduser('~/Downloads/glove.6B.{}d.txt'.format(EMB_DIM))
embedding_dict = utils.load_glove_to_dict(glove_path)

400000it [00:10, 36419.03it/s]


In [3]:
MAX_SEQ_LEN = 10 # The maximum input sequence length, if longer, truncate; if shorter, pad

complete_csv_path = './preprocessed.csv'
complete_df = pd.read_csv(complete_csv_path)

complete_vocab = utils.build_vocab(complete_df)

wvecs = utils.build_wvecs(embedding_dict, complete_vocab)

train_csv_path = './train.csv'
val_csv_path = './val.csv'
test_csv_path = './test.csv'

Vocabulary with 2169 items
2169


In [4]:
# Load dataset
train_dataset = utils.GarbageDataset(train_csv_path, complete_vocab, MAX_SEQ_LEN)
val_dataset = utils.GarbageDataset(val_csv_path, complete_vocab, MAX_SEQ_LEN)
test_dataset = utils.GarbageDataset(test_csv_path, complete_vocab, MAX_SEQ_LEN)

# Construct dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, collate_fn=utils.collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, collate_fn=utils.collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=utils.collate_fn)

In [5]:
class baseline_mlp(nn.Module):
    # Baseline mlp for garbage classification
    def __init__(self, embedding_dim, vocab_size, max_seq_length):
        super(baseline_mlp, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.fc1 = nn.Linear(max_seq_length * 10, 100)
        self.fc2 = nn.Linear(100, 10)
        self.out = nn.Linear(10, 4)
    
    def forward(self, sequence):
        batch_size = sequence.shape[0]
        embedded = self.embedding(sequence)
        embedded = embedded.view(batch_size, -1)
        z = self.fc1(embedded)
        z = self.fc2(z)
        z = self.out(z)
        return z


In [22]:
def eval(loader: DataLoader, model):
    """Evaluate the classification accuracy of model on the given dataset

    Args:
        loader (DataLoader): 
        model ([type])
    """
    correct_counts = 0
    with torch.no_grad():
        for batch in loader:
            input, target = batch
            input, target = input.to(device), target.to(device)
            pred = model(input).squeeze(1)
            pred_classes = torch.argmax(pred, dim=1)
            correct_counts += torch.sum(pred_classes == target)
    print(correct_counts.item() / len(loader.dataset))

In [23]:

use_cuda = True
device = torch.device("cuda:0" if use_cuda else "cpu") 

mlp = baseline_mlp(100, len(complete_vocab)).to(device)
mlp.embedding.weight.data.copy_(torch.from_numpy(np.array(wvecs)))
max_epoch = 100

optimizer = torch.optim.Adam(mlp.parameters(), lr=5e-4)

loss_fn = nn.CrossEntropyLoss()
loss_fn.to(device)

print('Start training')
for epoch in range(max_epoch):
    mlp.train()
    epoch_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input, target = batch
        input, target = input.to(device), target.to(device)
        pred = mlp(input).squeeze(1)
        loss = loss_fn(pred, target.long())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * target.shape[0]
    eval(val_loader, mlp)
    print('Epoch {} loss {}'.format(epoch, epoch_loss))



Start training
0.4300254452926209
Epoch 0 loss 3964.3510863780975
0.49872773536895676
Epoch 1 loss 3738.705544233322
0.5190839694656488
Epoch 2 loss 3409.8607709407806
0.5623409669211196
Epoch 3 loss 2948.29946577549
0.5877862595419847
Epoch 4 loss 2465.821175098419
0.5903307888040712
Epoch 5 loss 2039.1027629375458
0.6055979643765903
Epoch 6 loss 1686.3782294988632
0.6335877862595419
Epoch 7 loss 1402.8048535585403
0.6234096692111959
Epoch 8 loss 1183.7822397351265
0.6310432569974554
Epoch 9 loss 1025.7813694775105
0.6361323155216285
Epoch 10 loss 984.9664301276207
0.628498727735369
Epoch 11 loss 1014.6579702347517
0.628498727735369
Epoch 12 loss 796.1268595904112
0.6412213740458015
Epoch 13 loss 622.7976109683514
0.6234096692111959
Epoch 14 loss 530.6231648027897
0.6361323155216285
Epoch 15 loss 423.4408654868603
0.6335877862595419
Epoch 16 loss 360.396874088794
0.6412213740458015
Epoch 17 loss 320.8222685456276
0.6361323155216285
Epoch 18 loss 288.8390903659165
0.6361323155216285
Ep

In [26]:
eval(test_loader, mlp)

0.6423173803526449
